# 2D Model

In [1]:
import logging

# Set up the root logger to output DEBUG and higher level logs to console
logging.basicConfig(level=logging.INFO)

# Now your custom logger will output messages at the INFO level and above
logger = logging.getLogger(__name__)
logger.info('This is an info message')

INFO:__main__:This is an info message


In [2]:
from recognizer.utils.constants import DATASET_DIR, TARGET_TO_ENCODING
from recognizer.utils.utils import get_metadata_from_filename

In [3]:
import os

import pandas as pd 


targets = []
subjects = []
repetitions = []
files = []

for file in os.listdir(DATASET_DIR):
    if "left" in file:
        continue

    target, subject, repetition = get_metadata_from_filename(file)

    targets.append(target)
    subjects.append(subject)
    repetitions.append(repetition)
    files.append(str((DATASET_DIR / file).resolve()))


metadata = pd.DataFrame(
    data={
        "target": targets,
        "subject": subjects,
        "repetition": repetitions,
        "file": files,
    }
)

metadata["target_encoding"] = metadata["target"].map(TARGET_TO_ENCODING)

metadata.head()

,target,subject,repetition,file,target_encoding
0,064,002,001,/Users/facundopalavecino/Documents/DEV/ecd-tra...,63
1,035,010,003,/Users/facundopalavecino/Documents/DEV/ecd-tra...,34
2,033,009,003,/Users/facundopalavecino/Documents/DEV/ecd-tra...,32
3,026,002,004,/Users/facundopalavecino/Documents/DEV/ecd-tra...,25
4,023,005,002,/Users/facundopalavecino/Documents/DEV/ecd-tra...,22


### Train/Test split

In [4]:
import numpy as np

size = 1
replace = False
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]

testing_set = metadata.groupby(["target", "subject"], as_index=False).apply(fn)

testing_set.index = testing_set.index.droplevel(0)

training_set = metadata.loc[~metadata.index.isin(testing_set.index), :]

In [5]:
training_set = training_set.sample(frac=0.3)
testing_set = testing_set.sample(frac=0.3)

In [6]:
def transform(x):
    """Permutes the element to match the format expected by PyTorch: (C<channels>, H<height>, W<width>)"""
    # Transpose video from (T<frames>, Height, Width, Channels) to (Channels, Height, Width)
    return x.permute(2, 0, 1)


In [7]:
from recognizer.dataset.image_dataset import ImageDataset

training_dataset = ImageDataset(
    video_filenames=training_set["file"].values,
    labels=training_set["target_encoding"].values,
    transform=transform,
)

testing_dataset = ImageDataset(
    video_filenames=testing_set["file"].values,
    labels=testing_set["target_encoding"].values,
    transform=transform,
)

print(f"Training/testing set: ({len(training_dataset)}, {len(testing_dataset)})")

Training/testing set: (768, 192)


In [8]:
BATCH_SIZE = 64
NUM_CLASSES = 64
EPOCHS = 2

In [9]:
import torch 

from torch import nn

from recognizer.models.simple_2d import Simple2DCNN

model = Simple2DCNN(
    num_classes=NUM_CLASSES,
    batch_size=BATCH_SIZE,
)

train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle = False)

test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=BATCH_SIZE, shuffle = False)

loss_function = nn.CrossEntropyLoss()

learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)


## Training

In [10]:
from recognizer.trainers import Trainer

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    loss_function=loss_function,
    optimizer=optimizer,
    learning_rate=learning_rate,
    device=device
)

INFO:recognizer.trainers.trainer:Using device: mps


In [11]:
trainer.train(epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]INFO:recognizer.trainers.trainer:Beginning training...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 1...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 2...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 3...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 4...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 5...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 6...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 7...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 8...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 9...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 10...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 11...
INFO:recognizer.trainers.trainer:Epoch 1 | Batch 12...
12it [00:14,  1.24s/it]
INFO:recognizer.trainers.trainer:Epoch 1, Training Loss: 12.88
INFO:recognizer.trainers.trainer:Beginning of testing...
100%|██████████| 3/3 [00:03<00:00,  1.02s/it]
INFO:recognizer.trainers.trainer:Epoch 1: Test Accuracy: 0.00
 33%|